# Homework 2


Hồ Thanh Nhân - 21127122

---

[1]

# Hoeffding Inequality

### Import neccessary libraries

In [127]:
import numpy as np
from typing import List

In [128]:
N = 1000 # 1000 virtual fair coins
T = 10 # 10 times of flipping each coin independently
M = 100000 # 100,000 times of running the experiment
states = ['H', 'T'] # 2 states of flipping coin. Head and Tail
coins = {}

# Contants
EPSILON = 0.4 # large tolerance because sample size is small
E_OUT = 0.5 # law of large number

### Implement experiment for M time(s)

In [129]:
def experiments(M = 1):
    v_1: List[float] = []
    v_rand: List[float] = []
    v_min: List[float] = []
    for j in range(M):
        # Flipping 1000 virtual fair coins. Flip each coin independently 10 times.
        coins = np.random.choice(['H', 'T'], size=(N, T))

        # The first coin flipped
        c_1 = coins[0]

        # A coin chosen randomly from 1000 coins
        c_rand = coins[np.random.randint(N)]

        # c_min is the coin which had the minimum frequency of heads
        min_head_index = np.argmin(np.sum(coins == 'H', axis = 1))
        c_min = coins[min_head_index]

        # v_1, v_rand, v_min (the fraction of heads obtained for the 3 respective coins out of the 10 tosses)
        v_1.append(np.sum(c_1 == 'H') / T)
        v_rand.append(np.sum(c_rand == 'H') / T)
        v_min.append(np.sum(c_min == 'H') / T)

    return np.array(v_1), np.array(v_rand), np.array(v_min)

### Run the experiment 100,000 times

In [130]:
v_1_list, v_rand_list, v_min_list = experiments(M)

# Calculate average value for v_1, v_rand, v_min
v_1 = np.sum(v_1_list) / M
v_rand = np.sum(v_rand_list) / M
v_min = np.sum(v_min_list) / M

## 1. 

In [131]:
v_min

0.037738999999999995

Therefore, the closest answer is [b] 0.01

## 2.

[2]

Calculate the right side of Hoeffding Inequality:

In [132]:
hoeffding_bound = 2 * np.exp(-2 * EPSILON**2 * T)

Check coin(s) has a distribution of v that satisfies the (single-bin) Hoeffding Inequality

In [133]:
def satisfy_Hoeffding_Inequality(v):
    return np.sum(np.abs(v - E_OUT) > EPSILON) / M < hoeffding_bound

In [134]:
# Check v_1
print(satisfy_Hoeffding_Inequality(v_1_list))

# Check v_rand
print(satisfy_Hoeffding_Inequality(v_rand_list))

# Check v_min
print(satisfy_Hoeffding_Inequality(v_min_list))

True
True
False


Therefore, the answer is [d] c1 and crand

# Error and Noise

[3]

## 3.

There are 2 posibilities of error that h makes in approximating y:
- $h$ makes an error with probability $\mu$ but $y$ receives the correct value ($y = f(x)$) with probability $\lambda$.
- $h$ doesn't make error with probability ($1 - \mu$) but $y$ receives the incorrect value ($y \neq f(x)$) with probability ($1 - \lambda$).

In total, we get:

$(1 - \lambda) * (1 - \mu) + \lambda * \mu$

Therefore, the answer is [e] $(1 - \lambda) * (1 - \mu) + \lambda * \mu$

## 4.

We have: $P(error) = (1 - \lambda) * (1 - \mu) + \lambda * \mu$ (Q3)

$\Leftrightarrow P(error) = 1 - \mu - \lambda + 2 * \mu * \lambda$

$\Leftrightarrow P(error) = 1 - \lambda + \mu * (2 * \lambda - 1)$

If $\lambda = \frac{1}{2}$, we get $P(error) = \frac{1}{2}$ which is a constant value.

Therefore, with $\lambda = \frac{1}{2}$, the performance of h will be independent of $\mu$

The answer is [b] 0.5

# Linear Regression

### Import neccessary libraries

In [135]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [136]:
NUM_TRAIN_POINTS = 100
NUM_TEST_POINTS = 1000

### Function definitions

Generating `target_w`, the vector of parameters of $f$ function

In [137]:
def generate_target_w():
    """
    Generates target_w from two random, uniformly distributed points in [-1, 1] x [-1, 1].
    
    Returns
    -------
    target_w : numpy array, shape (3, 1) 
        The vector of parameters of f.
    """
    # Generate two points from a uniform distribution over [-1, 1]x[-1, 1]
    p1 = np.random.uniform(-1, 1, 2)
    p2 = np.random.uniform(-1, 1, 2)
    # Compute the target W from these two points
    target_w = np.array([p1[1] * p2[0] - p1[0] * p2[1], p2[1] - p1[1], p1[0] - p2[0]]).reshape((-1, 1))
    
    return target_w

Generating data set function

In [138]:
def generate_data(N, target_w):
    """
    Generates a data set by generating random inputs and then using target_w to generate the 
    corresponding outputs.
    
    Parameters
    ----------
    N : int
        The number of examples.
    target_w : numpy array, shape (3, 1) 
        The vector of parameters of f.
    
    Returns
    -------
    X : numpy array, shape (N, 3)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones.
    Y : numpy array, shape (N, 1)
        The vector of outputs.        
    """
    X = np.random.uniform(-1, 1, (N, 2))
    X = np.hstack((np.ones((N, 1)), X)) # Add 'ones' column
    Y = np.sign(np.dot(X, target_w))
    
    return X, Y

Linear Regression for training

In [139]:
def run_Linear_Regression_train(X, Y):
    model = LinearRegression().fit(X, Y.reshape(-1))
    E_in = np.sum(np.sign(model.predict(X)) != Y.reshape(-1)) / len(X)
    return model, E_in

Linear Regression for testing

In [140]:
def run_Linear_Regression_test(X_test,  Y_test, model):
    E_out = np.sum(np.sign(model.predict(X_test)) != Y_test.reshape(-1)) / len(X_test)
    return E_out

Main function

In [141]:
def main():
    num_runs = 1000
    avg_E_in = 0.0 # The average E_in
    avg_E_out = 0.0 # The average E_out
    for r in range(num_runs):
        # Generate target_w
        target_w = generate_target_w()
        
        # Generate training set
        X, Y = generate_data(NUM_TRAIN_POINTS, target_w)
        
        # Run Linear Regression to pick g and evaluate E_in
        model, E_in = run_Linear_Regression_train(X, Y)

        # Generate test set
        X_test, Y_test = generate_data(NUM_TEST_POINTS, target_w)
        
        # Test g and evaluate E_out
        E_out = run_Linear_Regression_test(X_test, Y_test, model)
        
        # Update average values of E_in and E_out
        avg_E_in += (E_in * 1.0 / num_runs)
        avg_E_out += (E_out * 1.0 / num_runs)
    
    # Print results
    print('avg_E_in = %f' % (avg_E_in))
    print('avg_E_out = %f' % (avg_E_out))

Run

In [142]:
main()

avg_E_in = 0.040500
avg_E_out = 0.049294


## 5. 

avg_E_in is closest to the answer [c] 0.01

## 6. 

avg_E_out is closest to the answer [c] 0.01

## 7.

PLA function

In [143]:
def h(w, x):
    return np.sign(np.dot(x, w))

In [144]:
def has_converged(X, Y, w):
    return np.array_equal(h(w, X), Y)

In [145]:
def run_PLA(X, Y, w):
    """
    Runs PLA.
    
    Parameters
    ----------
    X : numpy array, shape (N, 3)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones.
    Y : numpy array, shape (N, 1)
        The vector of outputs.
    
    Returns
    -------
    w : numpy array, shape (3, 1) 
        The vector of parameters of g.
    num_iterations : int
        The number of iterations PLA takes to converge.
    """
    iteration = 0

    N = X.shape[0]

    while True:
        mis_X = []
        mis_Y = []
        iteration += 1
        mix_id = np.random.permutation(N)
        for i in range(N):
            xi = X[mix_id[i], :]
            yi = Y[mix_id[i], 0]
            if h(w, xi)[0] != yi: # misclassified point
                mis_X.append(xi)
                mis_Y.append(yi)

        if len(mis_X) > 0:
            random_index = np.random.randint(0, len(mis_X))
            x_i = mis_X[random_index]
            y_i = mis_Y[random_index]
            w = w + (y_i * x_i).reshape(-1, 1)
        
        if has_converged(X, Y, w):
            break
        
    return w, iteration

Main function

In [170]:
def main(N):
    """
    Parameters
    ----------
    N : int
        The number of training examples.
    """
    num_runs = 1000
    avg_num_iterations = 0.0 # The average number of iterations PLA takes to converge
    for r in range(num_runs):
        # Generate target_w
        target_w = generate_target_w()
        
        # Generate training set
        X, Y = generate_data(N, target_w)

        # Generate weights using Linear Regression
        X_Dagger = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T)
        w_init = np.dot(X_Dagger, Y)
        
        # Run PLA to pick g
        w, num_iterations = run_PLA(X, Y, w_init)
        
        # Update average values
        avg_num_iterations += (num_iterations * 1.0 / num_runs)
    
    # Print results
    print('avg_num_iterations = %f' % (avg_num_iterations))

Run

In [171]:
main(N=10)

avg_num_iterations = 4.383000


avg_num_iterations is closest to the answer [a] 1

# Nonlinear Transformation

In [189]:
N = 1000
num_runs = 1000

Generate X

In [190]:
def genX():
    X = np.random.uniform(-1, 1, size=(N, 2))
    return X

Calculate Y from X

In [191]:
def f(x):
    return np.sign(x[0]**2 + x[1]**2 - 0.6).astype(int)

## 8

In [204]:
avg_E_in = 0.0 # The average E_in

for i in range(num_runs):
    # Generate X
    X = genX()
    # Simulate noise by flipping the sign of the ouput in a randomly selected 10% subset
    Y = np.apply_along_axis(f, 1, X)
    noisy_index = np.random.choice(N, int(N/10), replace=False)
    for i in noisy_index:
        Y[i] = np.negative(Y[i])

    X = np.insert(X, 0, 1, axis=1) # Add 'ones' column

    X_Dagger = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T)
    w = np.dot(X_Dagger, Y) # the weight

    # Make predictions from linear regression weights
    Y_predict = np.sign(np.dot(X, w))

    # E_in
    E_in = sum(Y_predict != Y) / N

    avg_E_in += (E_in * 1.0 / num_runs)

print('avg_E_in = %f' % (avg_E_in))

avg_E_in = 0.505019


avg_E_in is closest to the answer [d] 0.5

## 9.

In [207]:
avg_agree_a = 0.0 # The average probability of agreeing of [a]'s hypothesis
avg_agree_b = 0.0 # The average probability of agreeing of [b]'s hypothesis
avg_agree_c = 0.0 # The average probability of agreeing of [c]'s hypothesis
avg_agree_d = 0.0 # The average probability of agreeing of [d]'s hypothesis
avg_agree_e = 0.0 # The average probability of agreeing of [e]'s hypothesis

for i in range(num_runs):
    # Generate X
    X = genX()
    x1s = X[:, 0]
    x2s = X[:, 1]

    # Simulate noise by flipping the sign of the ouput in a randomly selected 10% subset
    Y = np.apply_along_axis(f, 1, X)
    noisy_index = np.random.choice(N, int(N/10), replace=False)
    for i in noisy_index:
        Y[i] = np.negative(Y[i])

    # training data into the nonlinear feature vector
    X = np.array([np.ones(N), x1s, x2s, x1s * x2s, x1s**2, x2s**2]).T

    X_Dagger = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T)
    w = np.dot(X_Dagger, Y) # the weight

    # Make predictions from linear regression weights
    Y_predict = np.sign(np.dot(X, w))

    # [a]
    w_a = np.array([-1, -0.05, 0.08, 0.13, 1.5, 1.5])
    Y_a = np.sign(np.dot(X, w_a))
    agree_a = sum(Y_a == Y_predict) / N
    avg_agree_a += (agree_a * 1.0 / num_runs)

    # [b]
    w_b = np.array([-1, -0.05, 0.08, 0.13, 1.5, 15])
    Y_b = np.sign(np.dot(X, w_b))
    agree_b = sum(Y_b == Y_predict) / N
    avg_agree_b += (agree_b * 1.0 / num_runs)

    # [c]
    w_c = np.array([-1, -0.05, 0.08, 0.13, 15, 1.5])
    Y_c = np.sign(np.dot(X, w_c))
    agree_c = sum(Y_c == Y_predict) / N
    avg_agree_c += (agree_c * 1.0 / num_runs)

    # [d]
    w_d = np.array([-1, -1.5, 0.08, 0.13, 0.05, 0.05])
    Y_d = np.sign(np.dot(X, w_d))
    agree_d = sum(Y_d == Y_predict) / N
    avg_agree_d += (agree_d * 1.0 / num_runs)

    # [e]
    w_e = np.array([-1, -0.05, 0.08, 1.5, 0.15, 0.15])
    Y_e = np.sign(np.dot(X, w_e))
    agree_e = sum(Y_e == Y_predict) / N
    avg_agree_e += (agree_e * 1.0 / num_runs)

print('avg_agree_a = %f' % (avg_agree_a))
print('avg_agree_b = %f' % (avg_agree_b))
print('avg_agree_c = %f' % (avg_agree_c))
print('avg_agree_d = %f' % (avg_agree_d))
print('avg_agree_e = %f' % (avg_agree_e))

avg_agree_a = 0.962303
avg_agree_b = 0.663177
avg_agree_c = 0.662642
avg_agree_d = 0.632305
avg_agree_e = 0.560567


avg_agree_a is the highest value. That means hypothesis [a] has the highest probability of agreeing on a randomly selected point. Therefore, the answer is [a] $g(x_1, x_2) = sign(−1 − 0.05x_1 + 0.08x_2 + 0.13x_1x_2 + 1.5x_1^2 + 1.5x_2^2)$


## 10.

In [209]:
avg_E_out = 0.0 # The average E_out

for i in range(num_runs):
    # Generate X
    X = genX()
    x1s = X[:, 0]
    x2s = X[:, 1]

    # Simulate noise by flipping the sign of the ouput in a randomly selected 10% subset
    Y = np.apply_along_axis(f, 1, X)
    noisy_index = np.random.choice(N, int(N/10), replace=False)
    for i in noisy_index:
        Y[i] = np.negative(Y[i])

    # training data into the nonlinear feature vector
    X = np.array([np.ones(N), x1s, x2s, x1s * x2s, x1s**2, x2s**2]).T

    w = np.array([-1, -0.05, 0.08, 0.13, 1.5, 1.5]) # the weight

    # Make predictions from linear regression weights
    Y_predict = np.sign(np.dot(X, w))

    # [a]
    E_out = sum(Y != Y_predict) / N
    avg_E_out += (E_out * 1.0 / num_runs)

print('avg_E_out = %f' % (avg_E_out))

avg_E_out = 0.143101


avg_E_out is closest to the answer [b] 0.1

References:

   [1] Github, Florian Peter, Caltech Machine Learning Homework # 2, last commit date: 30/04/2021, access date: 13/11/2023, https://github.com/workflow/caltech-machine-learning-homework/blob/master/HW2.ipynb

   [2] notebook.community, Homework files, access date: 13/11/2023, https://notebook.community/akhileshh/lfd-caltech/homework02

   [3] Github, Edgardo Deza, Homework 2, Problem 3 & Problem 4, last commit date: 11/10/2017, access date: 13/11/2023, https://github.com/homefish/edX_Learning_From_Data_2017/blob/master/homework_2/homework_2_problem_3_4_Error_and_Noise.ipynb